In [8]:
import os
import skimage.io as ski
import skimage.transform as skt
import numpy as np
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

X = []
y = []

image_size = 160

data_directory = "MRI1_Data/"

classes = os.listdir(data_directory)

for c in classes:
    folder = os.path.join(data_directory, c)
    for image in os.listdir(folder):
        load_image = ski.imread(os.path.join(folder,image))
        load_image = skt.resize(load_image, (image_size,image_size,3))
        X.append(load_image)
        y.append(c)

X = np.array(X)
y = np.array(y)

train_X, val_X, train_y, val_y = train_test_split(X,y, test_size = 0.1)
train_X, test_X, train_y, test_y = train_test_split(train_X,train_y, test_size = 0.2)

train_y_new = []
for i in train_y:
    train_y_new.append(classes.index(i))
train_y = train_y_new
train_y = tf.keras.utils.to_categorical(train_y)

test_y_new = []
for i in test_y:
    test_y_new.append(classes.index(i))
test_y = test_y_new
test_y = tf.keras.utils.to_categorical(test_y)

val_y_new = []
for i in val_y:
    val_y_new.append(classes.index(i))
val_y = val_y_new
val_y = tf.keras.utils.to_categorical(val_y)

idg = ImageDataGenerator(
    rotation_range = 30,
    rescale = 1./255.,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
    vertical_flip  = True
)

idg.fit(train_X)
idg.fit(test_X)
idg.fit(val_X)



In [5]:
shapes = {}

for i in X:

    if i.shape not in shapes:

        shapes[i.shape] = 1

    else:

        shapes[i.shape] += 1



In [6]:
shapes

{(160, 160, 3): 4034, (160, 160): 3301, (160, 160, 4): 4}

In [9]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras import Model

### MobileNet Tuned to alpha = 0.75, alpha is the width multiplier and controls the width of the network. With alpha at 0.75 we are creating a model with 75% of the filters that would normally be in each layer

In [9]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.75)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)


model_mobilenet_alpha1 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_alpha1.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_alpha1.fit(train_X, train_y, epochs = 10, validation_data = (test_X, test_y))

10626956/10626956 [==============================] - 2s 0us/step
Epoch 1/10
82/82 [==============================] - 104s 883ms/step - loss: 0.6454 - accuracy: 0.7511 - val_loss: 0.7080 - val_accuracy: 0.7489
Epoch 2/10
82/82 [==============================] - 69s 845ms/step - loss: 0.2528 - accuracy: 0.9131 - val_loss: 0.4081 - val_accuracy: 0.8453
Epoch 3/10
82/82 [==============================] - 69s 838ms/step - loss: 0.1240 - accuracy: 0.9671 - val_loss: 0.3200 - val_accuracy: 0.8897
Epoch 4/10
82/82 [==============================] - 69s 837ms/step - loss: 0.0663 - accuracy: 0.9862 - val_loss: 0.2624 - val_accuracy: 0.9142
Epoch 5/10
82/82 [==============================] - 70s 855ms/step - loss: 0.0415 - accuracy: 0.9931 - val_loss: 0.2694 - val_accuracy: 0.9051
Epoch 6/10
82/82 [==============================] - 68s 836ms/step - loss: 0.0361 - accuracy: 0.9931 - val_loss: 0.2272 - val_accuracy: 0.9280
Epoch 7/10
82/82 [==============================] - 68s 831ms/step - loss: 0

In [10]:
model_mobilenet_alpha1.evaluate(x = val_X, y = val_y)

21/21 [==============================] - 5s 224ms/step - loss: 0.2051 - accuracy: 0.9311


[0.2051081508398056, 0.93108731508255]

### MobileNet Tuned to alpha = 0.5, alpha is the width multiplier and controls the width of the network. With alpha at 0.5 we are creating a model with 50% of the filters that would normally be in each layer

In [10]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.5)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)

model_mobilenet_alpha2 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_alpha2.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_alpha2.fit(train_X, train_y, epochs = 10, validation_data = (test_X, test_y), verbose = 1)



Epoch 1/10


74/74 [==============================] - 50s 502ms/step - loss: 0.7445 - accuracy: 0.7077 - val_loss: 1.2619 - val_accuracy: 0.4558
Epoch 2/10
74/74 [==============================] - 39s 523ms/step - loss: 0.3406 - accuracy: 0.8796 - val_loss: 1.1989 - val_accuracy: 0.5646
Epoch 3/10
74/74 [==============================] - 37s 499ms/step - loss: 0.2305 - accuracy: 0.9200 - val_loss: 0.6370 - val_accuracy: 0.7619
Epoch 4/10
74/74 [==============================] - 38s 510ms/step - loss: 0.1434 - accuracy: 0.9583 - val_loss: 0.7318 - val_accuracy: 0.7296
Epoch 5/10
74/74 [==============================] - 38s 508ms/step - loss: 0.0936 - accuracy: 0.9800 - val_loss: 0.4137 - val_accuracy: 0.8537
Epoch 6/10
74/74 [==============================] - 38s 509ms/step - loss: 0.0667 - accuracy: 0.9851 - val_loss: 0.3548 - val_accuracy: 0.8724
Epoch 7/10
74/74 [==============================] - 39s 525ms/step - loss: 0.0515 - accuracy: 0.9894 - val_loss: 0.2801 - val_accuracy: 0.

In [ ]:
model_mobilenet_alpha2.evaluate(x = val_X, y = val_y)

21/21 [==============================] - 3s 125ms/step - loss: 0.2135 - accuracy: 0.9342


[0.21353574097156525, 0.9341500997543335]

### MobileNet Tuned to alpha = 0.25, alpha is the width multiplier and controls the width of the network. With alpha at 0.25 we are creating a model with 25% of the filters that would normally be in each layer

In [15]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.25)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)

model_mobilenet_alpha3 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_alpha3.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_alpha3.fit(train_X, train_y, epochs = 20, validation_data = (test_X, test_y), verbose = 1)

Epoch 1/20
82/82 [==============================] - 41s 278ms/step - loss: 0.8766 - accuracy: 0.6455 - val_loss: 0.9932 - val_accuracy: 0.5620
Epoch 2/20
82/82 [==============================] - 18s 220ms/step - loss: 0.4933 - accuracy: 0.8300 - val_loss: 0.7356 - val_accuracy: 0.7320
Epoch 3/20
82/82 [==============================] - 18s 214ms/step - loss: 0.3465 - accuracy: 0.8828 - val_loss: 0.5912 - val_accuracy: 0.7688
Epoch 4/20
82/82 [==============================] - 18s 216ms/step - loss: 0.2780 - accuracy: 0.9043 - val_loss: 1.1088 - val_accuracy: 0.6845
Epoch 5/20
82/82 [==============================] - 18s 217ms/step - loss: 0.2094 - accuracy: 0.9349 - val_loss: 0.6736 - val_accuracy: 0.7596
Epoch 6/20
82/82 [==============================] - 18s 217ms/step - loss: 0.1655 - accuracy: 0.9433 - val_loss: 0.4897 - val_accuracy: 0.8239
Epoch 7/20
82/82 [==============================] - 18s 220ms/step - loss: 0.1283 - accuracy: 0.9613 - val_loss: 0.4160 - val_accuracy: 0.8377

In [16]:
model_mobilenet_alpha3.evaluate(x = val_X, y = val_y)

21/21 [==============================] - 1s 61ms/step - loss: 0.3520 - accuracy: 0.9127


[0.3520335257053375, 0.9127105474472046]

### MobileNet Tuned to dropout = 0.01, dropout represents the dropout rate used for regularizing to combat overfitting of the training models.
_We used alpha = 0.5 since it has the best out of sample preferences of all the alpha parameters_

In [ ]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.5, dropout=0.01)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)

model_mobilenet_dropout1 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_dropout1.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_dropout1.fit(train_X, train_y, epochs = 10, validation_data = (test_X, test_y), verbose = 1)

Epoch 1/10
82/82 [==============================] - 70s 594ms/step - loss: 0.7094 - accuracy: 0.7293 - val_loss: 1.2200 - val_accuracy: 0.4717
Epoch 2/10
82/82 [==============================] - 41s 501ms/step - loss: 0.3360 - accuracy: 0.8783 - val_loss: 1.6674 - val_accuracy: 0.3583
Epoch 3/10
82/82 [==============================] - 40s 492ms/step - loss: 0.1975 - accuracy: 0.9391 - val_loss: 1.1516 - val_accuracy: 0.5988
Epoch 4/10
82/82 [==============================] - 40s 492ms/step - loss: 0.1259 - accuracy: 0.9678 - val_loss: 0.6989 - val_accuracy: 0.7397
Epoch 5/10
82/82 [==============================] - 44s 535ms/step - loss: 0.0810 - accuracy: 0.9809 - val_loss: 0.7341 - val_accuracy: 0.7596
Epoch 6/10
82/82 [==============================] - 44s 541ms/step - loss: 0.0546 - accuracy: 0.9900 - val_loss: 0.3963 - val_accuracy: 0.8668
Epoch 7/10
82/82 [==============================] - 54s 661ms/step - loss: 0.0374 - accuracy: 0.9935 - val_loss: 0.2913 - val_accuracy: 0.8897

In [20]:
model_mobilenet_dropout1.evaluate(x = val_X, y = val_y)

21/21 [==============================] - 3s 141ms/step - loss: 0.2347 - accuracy: 0.9204


[0.23469983041286469, 0.9203675389289856]

### Dropout 0.1

In [21]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.5, dropout=0.1)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)

model_mobilenet_dropout2 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_dropout2.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_dropout2.fit(train_X, train_y, epochs = 10, validation_data = (test_X, test_y), verbose = 1)

Epoch 1/10
82/82 [==============================] - 67s 534ms/step - loss: 0.7327 - accuracy: 0.7167 - val_loss: 1.2480 - val_accuracy: 0.5345
Epoch 2/10
82/82 [==============================] - 40s 492ms/step - loss: 0.3293 - accuracy: 0.8905 - val_loss: 0.9709 - val_accuracy: 0.5988
Epoch 3/10
82/82 [==============================] - 40s 488ms/step - loss: 0.2013 - accuracy: 0.9330 - val_loss: 0.6682 - val_accuracy: 0.7152
Epoch 4/10
82/82 [==============================] - 40s 483ms/step - loss: 0.1315 - accuracy: 0.9594 - val_loss: 0.4501 - val_accuracy: 0.8407
Epoch 5/10
82/82 [==============================] - 41s 496ms/step - loss: 0.0827 - accuracy: 0.9820 - val_loss: 0.4273 - val_accuracy: 0.8499
Epoch 6/10
82/82 [==============================] - 39s 481ms/step - loss: 0.0489 - accuracy: 0.9916 - val_loss: 0.2870 - val_accuracy: 0.9005
Epoch 7/10
82/82 [==============================] - 40s 482ms/step - loss: 0.0371 - accuracy: 0.9943 - val_loss: 0.2496 - val_accuracy: 0.9127

In [22]:
model_mobilenet_dropout2.evaluate(x = val_X, y = val_y)

21/21 [==============================] - 3s 128ms/step - loss: 0.2358 - accuracy: 0.9280


[0.2358420342206955, 0.9280245304107666]

### Dropout 0.2

In [23]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.5, dropout=0.2)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)


model_mobilenet_dropout3 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_dropout3.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_dropout3.fit(train_X, train_y, epochs = 10, validation_data = (test_X, test_y), verbose = 1)

Epoch 1/10
82/82 [==============================] - 74s 576ms/step - loss: 0.7237 - accuracy: 0.7286 - val_loss: 1.7130 - val_accuracy: 0.4472
Epoch 2/10
82/82 [==============================] - 40s 493ms/step - loss: 0.3191 - accuracy: 0.8924 - val_loss: 0.8099 - val_accuracy: 0.6508
Epoch 3/10
82/82 [==============================] - 41s 502ms/step - loss: 0.2065 - accuracy: 0.9338 - val_loss: 0.7298 - val_accuracy: 0.7152
Epoch 4/10
82/82 [==============================] - 41s 495ms/step - loss: 0.1390 - accuracy: 0.9598 - val_loss: 0.7197 - val_accuracy: 0.7228
Epoch 5/10
82/82 [==============================] - 40s 487ms/step - loss: 0.0925 - accuracy: 0.9786 - val_loss: 0.4044 - val_accuracy: 0.8331
Epoch 6/10
82/82 [==============================] - 40s 492ms/step - loss: 0.0582 - accuracy: 0.9900 - val_loss: 0.2540 - val_accuracy: 0.9096
Epoch 7/10
82/82 [==============================] - 40s 491ms/step - loss: 0.0468 - accuracy: 0.9908 - val_loss: 0.2181 - val_accuracy: 0.9250

In [ ]:
model_mobilenet_dropout3.evaluate(x = val_X, y = val_y)

### Dropout = 0.3

In [24]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.5, dropout=0.3)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)


model_mobilenet_dropout3 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_dropout3.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_dropout3.fit(train_X, train_y, epochs = 10, validation_data = (test_X, test_y), verbose = 1)

Epoch 1/10
82/82 [==============================] - 70s 523ms/step - loss: 0.7570 - accuracy: 0.7037 - val_loss: 1.0284 - val_accuracy: 0.5482
Epoch 2/10
82/82 [==============================] - 40s 487ms/step - loss: 0.3555 - accuracy: 0.8836 - val_loss: 0.5965 - val_accuracy: 0.7749
Epoch 3/10
82/82 [==============================] - 40s 486ms/step - loss: 0.2266 - accuracy: 0.9211 - val_loss: 0.5058 - val_accuracy: 0.8025
Epoch 4/10
82/82 [==============================] - 40s 483ms/step - loss: 0.1464 - accuracy: 0.9579 - val_loss: 0.4825 - val_accuracy: 0.8040
Epoch 5/10
82/82 [==============================] - 40s 490ms/step - loss: 0.1041 - accuracy: 0.9743 - val_loss: 0.3002 - val_accuracy: 0.8775
Epoch 6/10
82/82 [==============================] - 41s 505ms/step - loss: 0.0635 - accuracy: 0.9851 - val_loss: 0.2294 - val_accuracy: 0.9173
Epoch 7/10
82/82 [==============================] - 44s 536ms/step - loss: 0.0521 - accuracy: 0.9885 - val_loss: 0.3227 - val_accuracy: 0.8760

In [ ]:
model_mobilenet_dropout3.evaluate(x = val_X, y = val_y)

### Dropout 0.5

In [25]:

base_model = MobileNet(include_top=False, input_shape=(160, 160, 3),alpha = 0.5, dropout=0.5)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
preds = Dense(4,activation = "softmax")(x)


model_mobilenet_dropout4 = Model(inputs = base_model.input, outputs=preds)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_mobilenet_dropout4.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])

history = model_mobilenet_dropout4.fit(train_X, train_y, epochs = 10, validation_data = (test_X, test_y), verbose = 1)

Epoch 1/10
47/82 [================>.............] - ETA: 17s - loss: 0.9170 - accuracy: 0.6330

KeyboardInterrupt: 

In [ ]:
model_mobilenet_dropout4.evaluate(x = val_X, y = val_y)

# Testing on new dataset

In [11]:
new_dataset_X = []
new_dataset_y = []

image_size = 160

data_directory = "MRI2_Data/"

for c in classes:
    folder = os.path.join(data_directory, c)
    for image in os.listdir(folder):
        load_image = ski.imread(os.path.join(folder,image))
        load_image = skt.resize(load_image, (image_size,image_size))
        new_dataset_X.append(load_image)
        new_dataset_y.append(c)

new_dataset_X = np.array(X)
new_dataset_y = np.array(y)

print("Dataset loaded")

new_dataset_y_new = []
for i in new_dataset_y:
    new_dataset_y_new.append(classes.index(i))
new_dataset_y = new_dataset_y_new
new_dataset_y = tf.keras.utils.to_categorical(new_dataset_y)

idg.fit(new_dataset_X)

model_mobilenet_alpha2.evaluate(x = new_dataset_X, y = new_dataset_y)

Dataset loaded
103/103 [==============================] - 19s 178ms/step - loss: 0.0704 - accuracy: 0.9770


[0.0703960657119751, 0.9770290851593018]

In [13]:
new_dataset_y_preds = model_mobilenet_alpha2.predict(new_dataset_X)
new_dataset_y_preds